In [1]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SQLContext, SparkSession

In [2]:
import time
import shutil
import sys
import requests

In [3]:
def aggregate_tags_count(new_values, total_sum):
	return sum(new_values) + (total_sum or 0)

In [12]:
checkpoint_path = r"A PATH TO A NOT EXISTED DIRECTORY"
shutil.rmtree(checkpoint_path, ignore_errors=True)

sc = SparkContext("local[2]", "TwitterStreamApp")
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 2)
sqlContext = SQLContext(sc)
ssc.checkpoint(checkpoint_path)

dataStream = ssc.socketTextStream("127.0.0.1", 9009)

words = dataStream.flatMap(lambda line: line.split(" "))
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))

tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
tags_totals.pprint()


In [13]:
ssc.start()

-------------------------------------------
Time: 2023-01-21 04:49:14
-------------------------------------------

-------------------------------------------
Time: 2023-01-21 04:49:16
-------------------------------------------

-------------------------------------------
Time: 2023-01-21 04:49:18
-------------------------------------------

-------------------------------------------
Time: 2023-01-21 04:49:20
-------------------------------------------
('#IranRevolutionRT', 1)
('#مهسا_امینی', 2)
('#IRGCterroristsRT', 1)
('#IRGCterrorists@sibzaminizogali', 1)
('#IRGCterrorists', 1)
('#IranRevoIution', 2)
('#IR…RT', 1)
('#IRGCterrorists@ferediiiiiii1', 1)

-------------------------------------------
Time: 2023-01-21 04:49:22
-------------------------------------------
('#IranRevolutionRT', 1)
('#مهسا_امینی', 2)
('#IRGCterroristsRT', 1)
('#IRGCterrorists@sibzaminizogali', 1)
('#IRGCterrorists', 4)
('#IranRevoIution', 4)
('#IR…RT', 1)
('#IRGCterrorists@ferediiiiiii1', 1)
('#محمد_حسینی', 

In [14]:
ssc.stop()

-------------------------------------------
Time: 2023-01-21 04:49:30
-------------------------------------------
('#IranRevolutionRT', 1)
('#مهسا_امینی', 8)
('#IRGCterroristsRT', 5)
('#IRGCterrorists@sibzaminizogali', 1)
('#ps752', 1)
('#RaisiTerrorist', 1)
('#ToomajSalehi', 1)
('#HowHealthyIsToomajReally', 1)
('#Zahedan', 1)
('#Kurdistan', 1)
...

